### TensorFlow 基础

In [11]:
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
print('TensorFlow version: {0}'.format(tf.__version__))
# tensorboard --logdir=path/to/log-directory
# export TF_CPP_MIN_LOG_LEVEL=3
# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

TensorFlow version: 1.13.1


#### tf 查看设备

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11124729929341338649, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 3193713459
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14853285334234282729
 physical_device_desc: "device: 0, name: GeForce GTX 960M, pci bus id: 0000:02:00.0, compute capability: 5.0"]

#### tf 常量+环境式session+矩阵乘操作

In [4]:
matrix1 = tf.constant([[3., 3.]])
matrix2 = tf.constant([[2.], [2.]])
product = tf.matmul(matrix1, matrix2)
with tf.Session() as sess:
      print(sess.run(product))


[[12.]]


#### tf 变量定义

In [4]:
from tensorflow.keras import initializers

In [1]:
config={
    "initializer_range":0.02,
    
    "age_embedding_size":10,
    
    "sku_count":48639,
    "sku_embedding_size":100,
    "cate_count":1426,
    "cate_embedding_size":10,
    "brand_count":2814,
    "brand_embedding_size":20
    
}

In [ ]:
init_normal=initializers.TruncatedNormal(stddev=config['initializer_range'])
sku_embedding_w=tf.get_variable('sku_embedding_w1', shape=(config['sku_count'],config['sku_embedding_size']), initializer=init_normal)
cate_embedding_w=tf.get_variable('cate_embedding_w1', shape=(config['sku_count'],config['cate_embedding_size']), initializer=init_normal)
brand_embedding_w=tf.get_variable('brand_embedding_w1', shape=(config['sku_count'],config['brand_embedding_size']), initializer=init_normal)

In [22]:
sku_embedding_w=tf.Variable(init_normal(shape=(config['sku_count'],config['sku_embedding_size']),dtype=tf.float32),name='sku_embedding_w')
cate_embedding_w=tf.Variable(init_normal(shape=(config['sku_count'],config['cate_embedding_size']),dtype=tf.float32),name='sku_embedding_w')
brand_embedding_w=tf.Variable(init_normal(shape=(config['sku_count'],config['brand_embedding_size']),dtype=tf.float32),name='sku_embedding_w')

In [14]:
weights=tf.concat([sku_embedding_w,cate_embedding_w,brand_embedding_w],axis=1) 

In [15]:
weights

<tf.Tensor 'concat_2:0' shape=(48639, 130) dtype=float32>

#### tf 交互式session + 张量加操作

In [3]:
sess = tf.Session()
node1 = tf.constant(3.0, tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
node3 = tf.add(node1, node2)
print("node3: ", node3)
print("sess.run(node3): ",sess.run(node3))

node3:  Tensor("Add:0", shape=(), dtype=float32)
sess.run(node3):  7.0


#### tf占位变量，一般作为训练、测试数据的传入

In [4]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a+b   # + provides a shortcut for tf.add(a, b)
print(sess.run(adder_node,feed_dict= {a: 3, b:4.5}))
print(sess.run(adder_node,feed_dict={a: [1,3], b: [2, 4]}))
add_and_triple = adder_node * 3.
print(sess.run(add_and_triple, {a: 3, b:4.5}))

7.5
[3. 7.]
22.5


### 实现一个线性模型

#### 模型定义

In [5]:
# 模型定义
# Model parameters
W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)
# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W * x + b
y = tf.placeholder(tf.float32)
# loss
loss = tf.reduce_sum(tf.square(linear_model - y))  # sum of the squares
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


#### 模型训练

In [6]:
# training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]
# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)  # reset values to wrong
for i in range(1000):
    sess.run(train, {x: x_train, y: y_train})
# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})

print("W: %s b: %s loss: %s" % (curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [0.9999908] loss: 5.6999738e-11


#### 重置变量值，可以直接对模型参数进行干涉

In [8]:
fixW = tf.assign(W, [-1])
fixb = tf.assign(b, [1])
sess.run([fixW, fixb])  
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

0.0


### API

In [1]:
import keras.backend as K
import tensorflow as tf

Using TensorFlow backend.


#### 矩阵预算器（einsum）

In [18]:
s=K.constant([[12,12],[12,12]])
t=K.constant([[12,12,3],[12,12,3]])
t=K.reshape(t,(-1,2,3))

# 向量(2,2) * (1,2,3) 相乘运算 结果(1,2,3)
res=tf.einsum('ij,kjp->kip',s,t)
K.eval(res)

array([[[288., 288.,  72.],
        [288., 288.,  72.]]], dtype=float32)

In [13]:
s=K.constant([[12,12],[12,12]])
t=K.constant([[12,12,3],[12,12,3]])
K.eval(K.dot(s,t))

array([[288., 288.,  72.],
       [288., 288.,  72.]], dtype=float32)

#### batch dot

In [3]:
s=K.constant([[12],[12]])
t=K.constant([[12,12,3],[12,12,3]])
s=K.reshape(s,(-1,2,1))
t=K.reshape(t,(-1,2,3))

In [6]:
K.batch_dot(s,t,(1,1))

<tf.Tensor 'MatMul_1:0' shape=(1, 1, 3) dtype=float32>

In [5]:
K.eval(K.batch_dot(s,t,(1,1)))

array([[[288., 288.,  72.]]], dtype=float32)

In [76]:
K.eval(K.zeros((1,2,3)))

array([[[0., 0., 0.],
        [0., 0., 0.]]], dtype=float32)

#### 填充

In [3]:
s=K.constant([[12,12],[12,12]])
K.eval(tf.pad(s,tf.constant([[0,0], [0,10]])))

array([[12., 12.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [12., 12.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]],
      dtype=float32)

#### 动态填充
>* 将s张量(1,2,2) 填充至(1,2,10)
>* 动态计算还需填充的维度 k - y =8 ，其中y为张量，k为常量

In [10]:
s=K.constant([[12,12],[12,12]])
s=K.reshape(s,(-1,2,2))
k=10
x=tf.constant([[10.0]])
y=K.cast(K.max([1,K.min([k,K.flatten(tf.log1p(x))[0]  ])]),'int32' )
print(K.eval(y))

2


In [15]:
zeros=K.zeros(shape=(K.shape(s)[0],K.shape(s)[1],k-y))
K.concatenate([s,zeros],axis=2)

<tf.Tensor 'concat:0' shape=(1, 2, 10) dtype=float32>

In [42]:
behavior_feature=K.zeros(shape=(10,50,100))

In [49]:
from keras.layers import *

In [50]:
sku_pooling=AveragePooling1D(pool_size=2,name='pooling_sku',data_format='channels_first')(behavior_feature)

In [51]:
K.eval(K.shape(sku_pooling))

array([10, 50, 50])